M Kronig (23029039)
CSC-40098 Final Project
Analysis of posts from the CS Educators Stack Exchange website
(SS3) What questions are computer science eduators asking?

In [ ]:
#Test notebook is running ok
print('Hello world!')

Hello world!


In [ ]:
import pandas as pd
import numpy as np

# Testing tthe dataframe
data = {'A': [1, 2, 3], 'B': [4, 5, 6],}
df = pd.DataFrame(data)

print(df)


   A  B
0  1  4
1  2  5
2  3  6
